In [ ]:
import openai
import tracely
import evidently.ui.runner as service_runner

### Let's run evidently ui service in background

In [ ]:
runner = service_runner.EvidentlyUIRunner()

In [ ]:
runner.run()

### You can explore service directly in this notebook, or you can open it in the new tab using the button above

In [ ]:
runner.show_service()

### Now you can get workspace

In [ ]:
workspace = runner.get_workspace()

### Let's create a new project and show traces page

In [ ]:
project = workspace.create_project(name="Example project")
project_id = project.id

project_id

In [ ]:
runner.show_service(service_runner.ServicePage(project_id=project_id, project_list='traces'))

### So, you don't have any traces yet.
### Let's setup tracing using `tracely`

In [ ]:
tracely.init_tracing(
    address=runner.get_service_url(),
    api_key="",
    project_id=project_id,
    export_name="Tweets",
)

In [ ]:
export_id = tracely.get_info()['export_id']

### Let's generate some tweets using OpenAI LLM

In [ ]:
@tracely.trace_event()
def basic_tweet_generation(topic, model="gpt-3.5-turbo", instructions=""):
    response = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": instructions},
            {"role": "user", "content": f"Write a short paragraph about {topic}"}
        ]
    )
    return response.choices[0].message.content

@tracely.trace_event()
def generate_tweet(topic):
    return basic_tweet_generation(topic)

In [ ]:
interesting_topics = [
    "Artificial Intelligence and Consciousness",
    "Quantum Computing Applications",
    "Space Exploration and Exoplanets",
    "Climate Change Solutions",
    "Neuroscience and Brain Mapping",
    "Ancient Civilizations and Lost Cities",
    "Renewable Energy Revolution",
    "Gene Editing and CRISPR",
    "Virtual Reality and the Metaverse",
    "Ocean Exploration and Deep Sea Life",
    "Cryptocurrency and Blockchain",
    "Meditation and Mindfulness Science",
    "Robotics and Human-Robot Interaction",
    "Food Science and Future Nutrition",
    "Urban Planning and Smart Cities",
]

In [ ]:
for topic in interesting_topics:
    generate_tweet(topic)

### Now we can explore traces in the UI

In [ ]:
runner.show_service(page=service_runner.ServicePage(project_id=project_id, export_id=export_id))

### And load this tracing data as dataset to this notebook

In [ ]:
dataset = workspace.load_dataset(dataset_id=export_id)

In [ ]:
dataset.as_dataframe()